<a href="https://colab.research.google.com/github/GustaveRw/3D-Flip-Card/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.2 MB/s 
     |████████████████████████████████| 5.8 MB 70.3 MB/s 
     |████████████████████████████████| 118 kB 74.5 MB/s 
     |████████████████████████████████| 212 kB 61.2 MB/s 
     |████████████████████████████████| 132 kB 57.6 MB/s 
     |████████████████████████████████| 182 kB 66.9 MB/s 
     |████████████████████████████████| 127 kB 72.4 MB/s 
     |████████████████████████████████| 7.6 MB 39.8 MB/s 
     |████████████████████████████████| 1.3 MB 59.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("<PATH>")
!ls

 data				    pt_save_pretrained	     translation.ipynb
 mt_save_pretrained		    Summarization.ipynb
 opus-mt-rw-en-finetuned-RW-to-EN  'text generation.ipynb'


In [ ]:
import pandas as pd
data = pd.read_csv('data/kinyarwanda-english-corpus2.tsv.txt',sep = '\t',encoding='cp1252',header=None,names=['RW','EN'])
data.tail(10)

,RW,EN
25005,Sinifuzaga gukora nkererewe ariko shobuja yamb...,I didn't want to to do I'm late however maste...
25006,Nta gahunda mfite y'ejo. Ngiye kubifata byoros...,No. plan i have of tomorrow . I am to take it ...
25007,Ntabwo mfite amafaranga ahagije yo kwishyura i...,I don't i have money enough yo to pay activiti...
25008,Sinzi niba numva neza ibyo ugerageza kuvuga.,I don't know if i feel well that you try to sp...
25009,Sinzi niba nzashobora kwitabira inama y'ejo.,I don't know if i can to participate advice of...
25010,Ntabwo nkunda igitabo gishya dukoresha murwego...,I don't i like book new we use at the level of...
25011,Sinshaka kurya muri iyo resitora. Ibiryo birab...,I don't want to to eat in salt restaurant . Fo...
25012,Ntabwo ntekereza ko nzanyura muriyi mirimo yos...,I don't i think no i will go through in this j...
25013,Ntabwo ntekereza ko iyi modoka ishaje izagera ...,I don't i think no this one car old will arriv...
25014,Ntabwo ntekereza ko dushobora kwitega ko hari ...,I don't i think no we can to expect no days an...


In [ ]:

data = data.dropna(subset=["EN"])
data = data.drop_duplicates(subset=["RW","EN"], keep='last')
data.describe()

,RW,EN
count,23490,23490
unique,23311,23259
top,Hari ikintu nkwiye kumenya?,That's the decent thing to do and it should en...
freq,3,10


In [ ]:
from datasets import Dataset, load_dataset, DatasetDict, load_metric

per80 = round ((len(data) * 80)/100)
training_dataset = Dataset.from_pandas(data[:per80])
testing_dataset =  Dataset.from_pandas(data[:-per80])

dataset = DatasetDict()
 
dataset['train'] = training_dataset
dataset['test'] = testing_dataset
 
dataset

DatasetDict({
    train: Dataset({
        features: ['RW', 'EN', '__index_level_0__'],
        num_rows: 18792
    })
    test: Dataset({
        features: ['RW', 'EN', '__index_level_0__'],
        num_rows: 4698
    })
})

In [ ]:
dataset['train'][1]

{'RW': 'Waba uzi umubare wabantu bitabiriye kubyina icyumweru gishize? ',
 'EN': 'Do you know the number of people who attended the dance last week?',
 '__index_level_0__': 1}

In [ ]:
metric = load_metric("sacrebleu")

<ipython-input-20-aede95515358>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-rw-en")

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/810k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/812k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [21, 11757, 2, 68, 48, 189, 156, 17761, 226, 3052, 93, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "RW"
target_lang = "EN"

def preprocess_function(examples):
    inputs = [ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
dataset['train'][:3]

{'RW': ['Ababyeyi babo bamaze gupfa basogokuru barabareze. ',
  'Waba uzi umubare wabantu bitabiriye kubyina icyumweru gishize? ',
  'Jya kuryama Ugomba kubyuka kare ejo mugitondo. '],
 'EN': ['After their parents died their grandparents raised them.',
  'Do you know the number of people who attended the dance last week?',
  'Go to bed You have to wake up early tomorrow morning.'],
 '__index_level_0__': [0, 1, 2]}

In [ ]:
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[3291, 479, 1232, 1765, 61, 3938, 1663, 8942, 15145, 12528, 3, 0], [11604, 2161, 2394, 52, 214, 12984, 14963, 13915, 20910, 16, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[669, 85, 532, 1530, 85, 19331, 2612, 121, 3, 0], [417, 33, 357, 5, 580, 8, 130, 90, 3869, 5, 17823, 763, 2224, 16, 0]]}

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "Helsinki-NLP/opus-mt-rw-en"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/303M [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: __index_level_0__, RW, EN. If __index_level_0__, RW, EN are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18649
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1166
  Number of trainable parameters = 75055104


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.081200,1.721858,33.733200,12.030500


Saving model checkpoint to opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-500
Configuration saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-500/config.json
Model weights saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-500/special_tokens_map.json
Saving model checkpoint to opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-1000
Configuration saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-1000/config.json
Model weights saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-1000/tokenizer_config.json
Special tokens file saved in opus-mt-rw-en-finetuned-RW-to-EN/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel

TrainOutput(global_step=1166, training_loss=2.1782943175955456, metrics={'train_runtime': 264.6098, 'train_samples_per_second': 70.477, 'train_steps_per_second': 4.406, 'total_flos': 81120641089536.0, 'train_loss': 2.1782943175955456, 'epoch': 1.0})

In [ ]:
pt_save_directory = "./mt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

tokenizer config file saved in ./mt_save_pretrained/tokenizer_config.json
Special tokens file saved in ./mt_save_pretrained/special_tokens_map.json
Configuration saved in ./mt_save_pretrained/config.json
Model weights saved in ./mt_save_pretrained/pytorch_model.bin


In [ ]:
pt_model = AutoModelForSeq2SeqLM.from_pretrained("./mt_save_pretrained")
tokenizer = AutoTokenizer.from_pretrained("./mt_save_pretrained")

loading configuration file ./mt_save_pretrained/config.json
Model config MarianConfig {
  "_name_or_path": "./mt_save_pretrained",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      60383
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 60383,
  "decoder_vocab_size": 60384,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LAB

In [ ]:
from transformers import pipeline
translation = pipeline(task="translation", model=pt_model, tokenizer=tokenizer)

In [ ]:
translation("Yongeye guhamagarwa mu 2021 hitegurwa Euro 2020.")

[{'translation_text': 'He was again invited to the 202th of the 20th century.'}]